# CodeExecutor


### Local Executor

`human_input_mode="ALWAYS"` 是为了手动验证正在执行代码的安全性（这是本地环境运行，有可能代码会破坏环境）


In [1]:
API_KEY = ""
BASE_URL = "https://open.bigmodel.cn/api/paas/v4/"

llm_config = {"model": "glm-4", "api_key": API_KEY, "base_url":BASE_URL}
import tempfile

from autogen import ConversableAgent

In [2]:
import tempfile

from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor

# Create a temporary directory to store the code files.
# temp_dir = tempfile.TemporaryDirectory()

# Create a local command line code executor.
executor = LocalCommandLineCodeExecutor(
    timeout=10,  # Timeout for each code execution in seconds.
    # work_dir=temp_dir.name,  # Use the temporary directory to store the code files.
    work_dir='./coding/local' # 指定目录
)

# Create an agent with code executor configuration.
code_executor_agent = ConversableAgent(
    "code_executor_agent",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the local command line code executor.
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety.
)

In [3]:
# 运行后直接回车会自动执行，否则 `message_with_code_block` 的值会变成输入的信息
message_with_code_block = """This is a message with code block.
The code block is below:
```python
import numpy as np
import matplotlib.pyplot as plt
x = np.random.randint(0, 100, 100)
y = np.random.randint(0, 100, 100)
plt.scatter(x, y)
plt.savefig('scatter.png')
print('Scatter plot saved to scatter.png')
```
This is the end of the message.
"""

# Generate a reply for the given code.
reply = code_executor_agent.generate_reply(messages=[{"role": "user", "content": message_with_code_block}])
print(reply)


>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
exitcode: 0 (execution succeeded)
Code output: Scatter plot saved to scatter.png



In [5]:
import os

print(os.listdir(temp_dir.name))
# We can see the output scatter.png and the code file generated by the agent.

['tmp_code_e24bf32d4a21990fb9e4b5eb889ebe5a.py', 'scatter.png']


### Docker Executor

In [2]:
from autogen.coding import DockerCommandLineCodeExecutor

# Create a temporary directory to store the code files.
# temp_dir = tempfile.TemporaryDirectory()

# Create a Docker command line code executor.
executor = DockerCommandLineCodeExecutor(
    image="python:3.12-slim",  # Execute code using the given docker image name.
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir="./coding/docker",  # Use the temporary directory to store the code files.
)

# Create an agent with code executor configuration that uses docker.
code_executor_agent_using_docker = ConversableAgent(
    "code_executor_agent_docker",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the docker command line code executor.
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety.
)

# When the code executor is no longer used, stop it to release the resources.
# executor.stop()

In [4]:
# 运行后直接回车会自动执行，否则 `message_with_code_block` 的值会变成输入的信息
message_with_code_block = """This is a message with code block.
The code block is below:
```python
import numpy as np
import matplotlib.pyplot as plt
x = np.random.randint(0, 100, 100)
y = np.random.randint(0, 100, 100)
plt.scatter(x, y)
plt.savefig('scatter.png')
print('Scatter plot saved to scatter.png')
```
This is the end of the message.
"""

# Generate a reply for the given code.
reply = code_executor_agent_using_docker.generate_reply(messages=[{"role": "user", "content": message_with_code_block}])
print(reply)


>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
exitcode: 1 (execution failed)
Code output: Traceback (most recent call last):
  File "/workspace/tmp_code_e24bf32d4a21990fb9e4b5eb889ebe5a.python", line 1, in <module>
    import numpy as np
ModuleNotFoundError: No module named 'numpy'



In [14]:
import os

print(os.listdir(temp_dir.name))

[]


## Use CodeExecutor in Conversation

In [3]:
# The code writer agent's system message is to instruct the LLM on how to use
# the code executor in the code executor agent.
code_writer_system_message = """你是一个乐于助人的人工智能助手。
使用你的编码和语言技能解决任务。
在以下情况下，建议用户执行 python 代码（在 python 编码块中）或 shell 脚本（在 sh 编码块中）。
1. 当你需要收集信息时，使用代码输出你需要的信息，例如，浏览或搜索网页、下载/阅读文件、打印网页或文件的内容、获取当前日期/时间、检查操作系统。在打印了足够的信息并且任务已准备好基于你的语言技能解决后，你就可以自己解决任务了。
2. 当你需要用代码执行某些任务时，使用代码执行任务并输出结果。巧妙地完成任务。
如果需要，请逐步解决任务。如果没有提供计划，请先解释你的计划。明确哪一步使用代码，哪一步使用你的语言技能。
使用代码时，你必须在代码块中指明脚本类型。用户不能提供任何其他反馈或执行除执行你建议的代码之外的任何其他操作。用户无法修改您的代码。因此，不要建议需要用户修改的不完整代码。如果代码块不打算由用户执行，请不要使用代码块。
如果您希望用户在执行代码之前将其保存在文件中，请将 #filename:  放在代码块的第一行。不要在一个响应中包含多个代码块。不要要求用户复制和粘贴结果。相反，在相关时使用“print”函数进行输出。检查用户返回的执行结果。
如果结果表明存在错误，请修复错误并再次输出代码。建议使用完整代码，而不是部分代码或代码更改。如果无法修复错误，或者即使代码成功执行后任务仍未解决，请分析问题，重新审视您的假设，收集您需要的其他信息，并想出另一种尝试的方法。
当您找到答案时，请仔细验证答案。如果可能，请在您的回复中包含可验证的证据。
当一切完成后，最后回复“TERMINATE”。
"""

code_writer_agent = ConversableAgent(
    "code_writer_agent",
    system_message=code_writer_system_message,
    llm_config=llm_config,
    code_execution_config={"executor":executor},  # Turn off code execution for this agent.
    human_input_mode="NEVER",   # 前面的 code
)

In [4]:
chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message="写一段Python代码查询当前日期和时间",
)

code_executor_agent (to code_writer_agent):

写一段Python代码查询当前日期和时间

--------------------------------------------------------------------------------
code_writer_agent (to code_executor_agent):

以下是一个Python代码片段，用于查询当前的日期和时间并打印出来。

```python
#filename: current_datetime.py

from datetime import datetime

# 获取当前日期和时间
now = datetime.now()

# 打印当前日期和时间
print("当前日期和时间：", now)
```

请将上面的代码保存到一个文件中，并执行它，以获取当前的日期和时间。如果需要以特定的格式显示日期和时间，可以修改打印语句中的格式。

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
code_executor_agent (to code_writer_agent):

exitcode: 0 (execution succeeded)
Code output: 当前日期和时间： 2024-08-27 17:01:57.347432


--------------------------------------------------------------------------------
[autogen.oai.client: 08-27 17:02:00] {329} WARNING - Model glm-4 is not found. The cost will be 0. In your config_list, add field {"price

现在尝试一个更复杂的案例，设计到 Web 查询

In [8]:
import datetime

today = datetime.datetime.now().strftime("%Y-%m-%d")
chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message=f"Today is {today}. Write Python code to plot TSLA's and META's "
    "stock price gains YTD, and save the plot to a file named 'stock_gains.png'.",
)

code_executor_agent (to code_writer_agent):

Today is 2024-08-27. Write Python code to plot TSLA's and META's stock price gains YTD, and save the plot to a file named 'stock_gains.png'.

--------------------------------------------------------------------------------
code_writer_agent (to code_executor_agent):

To plot the stock price gains of TSLA (Tesla) and META (Meta Platforms) YTD (Year-to-Date), we would typically need historical stock price data for both stocks. This data can be fetched from various financial APIs or from a file if it's been pre-downloaded. However, since I cannot directly access external data sources or the internet, I will outline a hypothetical approach using pseudo-code, assuming you have the necessary data in a CSV file named `stock_data.csv` with columns `Date`, `TSLA`, and `META`.

Here is the plan:
1. Read the stock data from a CSV file.
2. Calculate the YTD gains for both TSLA and META.
3. Plot the gains using a bar chart.
4. Save the plot to a file nam